In [ ]:
!pip install tensorflow[and-cuda]==2.15.1

In [ ]:
# To make sure we are using gpu not cpu
import tensorflow as tf

tf.config.list_physical_devices('GPU')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten , Dropout , BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, CSVLogger, EarlyStopping

from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
tf.__version__

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:

train = datagen.flow_from_directory('/kaggle/input/breast-mammography-private/train', target_size=(224, 224), class_mode='binary', batch_size=256)
# load and iterate validation dataset
# val = datagen.flow_from_directory('/kaggle/input/breast-mammography-ori/public/val', target_size=(224, 224), class_mode='binary', batch_size=16)
# load and iterate test dataset
test = datagen.flow_from_directory('/kaggle/input/breast-mammography-private/test', target_size=(224, 224), class_mode='binary', batch_size=8)

In [ ]:
STEP_SIZE_TRAIN = train.n//train.batch_size
STEP_SIZE_VAL = test.n//test.batch_size

In [ ]:
steps = test.n//test.batch_size

test.reset()
X_test, y_test = [], []
for i in range(steps+1):
    a , b = test.next()
    X_test.extend(a) 
    y_test.extend(b)

In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)
print(X_test.shape, y_test.shape)

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
def get_compiled_model():
    backbone2 = VGG16(input_shape = (224, 224, 3), include_top=False, weights='imagenet')
    backbone2.trainable = True
    set_trainable = False
    for layer in backbone2.layers:
        if layer.name == 'block4_conv1':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    VGG_model_2 = Sequential()
    VGG_model_2.add(backbone2)
    VGG_model_2.add(GlobalAveragePooling2D())
    VGG_model_2.add(BatchNormalization())
    VGG_model_2.add(Dropout(0.5))
    VGG_model_2.add(Dense(1, activation='sigmoid'))
    
    VGG_model_2.compile(
            loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
            metrics=['accuracy']
        )
    
    return VGG_model_2
    

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

In [ ]:
with strategy.scope():

    model = get_compiled_model()

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, 
                    steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=test,
                    validation_steps=STEP_SIZE_VAL,
                    epochs=10)


In [ ]:
score2 = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score2[0]} / Test accuracy: {score2[1]}')

In [ ]:
y_pred_prob_3 = model.predict(X_test)

In [ ]:
threshold = 0.5
y_pred_3 = np.where(y_pred_prob_3 > threshold, 1,0)
y_pred_3.squeeze()

In [ ]:
sns.set(rc={'figure.figsize':(7.7,6.27)})

sns.heatmap(confusion_matrix(y_test,y_pred_3),cmap=plt.cm.Blues,annot=True,annot_kws={"size": 32}, fmt='g')
plt.xticks([0.50,1.50], ['no cancer (0)','cancer (1)'], fontsize=20)
plt.yticks([0.50,1.50],['no cancer (0)','cancer (1)'], fontsize=20)

plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.title('Confusion Metrix for Breast Cancer')

In [ ]:
print(classification_report(y_test, y_pred_3, target_names = ['No cancer (Class 0)','cancer (Class 1)']))